In [ ]:
from IPython.display import clear_output
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import re
import dropbox
import sys
import pandas as pd
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
import logging
import datetime
import glob
import json
import scipy
import matplotlib as mpl

from subprocess import call
from tifffile import imwrite
from tqdm import tqdm
from pathlib import Path
from amftrack.util.dbx import (
    upload_folder,
    download,
    read_saved_dropbox_state,
    save_dropbox_state,
    load_dbx,
    get_dropbox_folders_prince,
    get_dropbox_video_folders,
    download_video_folders_drop,
    download_analysis_folders_drop,
)
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer
from amftrack.pipeline.launching.run_super import run_parallel_flows
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plot_data import (
    plot_summary,
    save_raw_data,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.high_mag_analysis import (
    HighmagDataset,
    VideoDataset,
    EdgeDataset,
    index_videos_dropbox_new,
    analysis_run,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.kymo_class import (
    KymoVideoAnalysis,
    KymoEdgeAnalysis,
)
from amftrack.util.dbx import (upload
)
%load_ext autoreload
%autoreload 2
%matplotlib widget

from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
    load_graphs,
    Edge,
    Node,

)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    reconstruct_image_from_general,
    plot_full,
    plot_edge_color_value,
)
from amftrack.transport.align_video_network import identify_nodes
logging.basicConfig(stream=sys.stdout, level=logging.debug)
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
mpl.rcParams["figure.dpi"] = 100
from amftrack.pipeline.functions.transport_processing.high_mag_videos.register_videos import *
from amftrack.pipeline.functions.transport_processing.high_mag_videos.loading import load_video_dataset
from amftrack.pipeline.functions.transport_processing.high_mag_videos.add_BC import *

from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
import networkx as nx


In [ ]:
videos_folder = "/projects/0/einf914/videos/"

analysis_folder = "/projects/0/einf914/analysis_videos/CocoTransport/"
analysis_folder_root = "/projects/0/einf914/analysis_videos/"
directory_targ = '/projects/0/einf914/transport/'


In [ ]:
indexes = {
"20230901_Plate310" : 20,
"20230902_Plate310" : 33,
"20230903_Plate310" : 42,
"20230904_Plate310" : 52,
"20230905_Plate310" : 64,
"20230906_Plate310" : 73,
}
indexes = {
"20230810_Plate441" : "20230810_1005_Plate14",
"20230811_Plate441" : "20230811_1605_Plate14",
"20230812_Plate441" : "20230812_1006_Plate14",
"20230813_Plate441" : "20230812_1618_Plate14",
}
indexes = {
"20230813_Plate449" : "20230813_1606_Plate10",
"20230814_Plate449" : "20230814_1019_Plate10",
"20230815_Plate449" : "20230815_1021_Plate10",
"20230816_Plate449" : "20230816_1027_Plate10",
}

In [ ]:
plate_id = "449_20230807"
# plate_id_video = "20230814_Plate449"
# data_obj = load_video_dataset(plate_id_video, videos_folder, analysis_folder, analysis_folder_root)

In [ ]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)
folders = all_folders.loc[all_folders["unique_id"] == plate_id]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
folders = folders.sort_values(by="datetime")

In [ ]:

folders = folders.sort_values(by="datetime")

exp = Experiment(directory_targ)
# i = indexes[plate_id_video]
# i = np.where(folders['folder'] == indexes[plate_id_video])[0][0]
selection = folders[folders['folder'].isin(indexes.values())]
# i = 10
# exp.load(folders.iloc[i : i + 2], suffix="_labeled")
exp.load(selection, suffix="_labeled")

for t in range(exp.ts):
    exp.load_tile_information(t)
    G = exp.nx_graph[t]
    components = nx.connected_components(G)

    # Find the largest connected component
    largest_component = max(components, key=len)

    # Create a new graph representing the largest connected component
    largest_component_graph = G.subgraph(largest_component)
    exp.nx_graph[t] = largest_component_graph
    exp.save_location = ""

    load_study_zone(exp)

# load_graphs(exp, directory_targ,indexes = [0])



In [ ]:
t=0
edges = get_all_edges(exp, t)
nodes = get_all_nodes(exp, t)
Vmax= 1
def get_weight(node):
    weight = 0
    for edge in node.edges(t):
        weight+= Vmax*np.pi*edge.width(t)/2
    return(weight)

def get_shortest_path_edges(node2,shortest):
    exp = node1.experiment
    nodes = shortest[node2.label]
    edges = []
    for i in range(len(nodes)-1):
        nodea = Node(nodes[i],exp)
        nodeb = Node(nodes[i+1],exp)
        edges.append(Edge(nodea,nodeb,exp))
    return(edges)

In [ ]:
nodes_sink= [
    node
    for node in nodes
    if not is_in_ROI_node(node, t)
]
nodes_source= [
    node
    for node in nodes
    if is_in_ROI_node(node, t)
]

In [ ]:
nodes_sink= [
    node
    for node in nodes
    if not is_in_ROI_node(node, t)
]
def find_lowest_nodes(nodes,t):
    positions = [node.pos(t) for node in nodes]

    # 1. Find the range of x positions
    min_x = min(positions, key=lambda p: p[1])[1]
    max_x = max(positions, key=lambda p: p[1])[1]

    # 2. Calculate the size of each part
    part_size = (max_x - min_x) / 10

    # 3. Group nodes by x range and select the node with the lowest y in each group
    selected_nodes = []
    for i in range(10):
        start_x = min_x + i * part_size
        end_x = start_x + part_size

        # Find nodes within the current x range
        nodes_in_range = [node for node in nodes if start_x <= node.pos(t)[1] < end_x]

        # 4. Select the node with the lowest y in the current range
        if nodes_in_range:
            lowest_y_node = max(nodes_in_range, key=lambda p: p.pos(t)[0])
            selected_nodes.append(lowest_y_node)
    return(selected_nodes)
nodes_sink = find_lowest_nodes(nodes_sink,t)
# nodes_sink

In [ ]:
t = 0
nodes = get_all_nodes(exp, t)

nodes= [
    node
    for node in nodes
    if is_in_ROI_node(node, t) and node.degree(t)==1 and len(node.ts())>1
]
ax = plot_full(exp, t, nodes=nodes,downsizing = 10,node_size = 10)


In [ ]:

edge_flux = {edge : 0 for edge in edges}
for node1 in nodes_sink:
    shortest = nx.single_source_dijkstra_path(exp.nx_graph[t],node1.label,weight="weight")
    for node2 in nodes_source:
        w = get_weight(node2)
        path = get_shortest_path_edges(node2,shortest)
        for edge in path:
            edge_flux[edge]+=w/len(nodes_sink)
    # break

In [ ]:
from matplotlib import cm
def get_radius2(edge,t):
    if edge.width(t)>2:
        return(edge.width(t)/2)
    else:
        return(1)
vmax = 2500
vmin = 0
plot_edge_color_value(
    exp,
    t,
    lambda edge: edge_flux[edge]/get_radius2(edge,t)**2,
    cmap=cm.get_cmap("viridis", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=False,
    dilation=10,
    figsize = (12,7)
)

In [ ]:
node1

In [ ]:
t = 2
begin,end = 1415372, 1571210


node1 = Node(begin,exp).get_pseudo_identity(t).label
node2 = end
# nodes = [Node(node1,exp),Node(node2,exp)]
nodes = nx.shortest_path(exp.nx_graph[t],node1,node2,weight = "weight")
nodes = [Node(node,exp) for node in nodes]
edges = [Edge(nodes[i],nodes[i+1],exp) for i in range(len(nodes)-1)]

In [ ]:
lengths = [edge.length_um(t) for edge in edges]
abciss = np.cumsum(lengths)
widths = np.array([edge.width(t) for edge in edges])
V=3e-9
phis = np.array([edge_flux[edge] for edge in edges])*V #ug.h-1

phis = phis * 1e-9 #dm**3/h
phis = phis * 1e15 #um**3/h


fig,ax = plt.subplots()
ax2 = ax.twinx()
# ax.scatter(abciss,phis/widths**2)
ax.scatter(abciss,phis)

# ax2.scatter(abciss,widths,color = "red")

In [ ]:
Node(1644494,exp).get_pseudo_identity(0)

In [ ]:
def get_abcisse(edge,begin,end,t):
    begin = Node(begin,begin.experiment).get_pseudo_identity(t).label

    nodes = nx.shortest_path(exp.nx_graph[t],begin,end,weight = "weight")
    nodes = [Node(node,exp) for node in nodes]
    edges = [Edge(nodes[i],nodes[i+1],exp) for i in range(len(nodes)-1)]
    lengths = [edge.length_um(t) for edge in edges]
    abciss = np.cumsum(lengths)
    if edge in edges:
        i = edges.index(edge)
        return(abciss[i])
    else:
        return(-1)

In [ ]:
get_abcisse(edges[10],node1,node2)

In [ ]:
# nodes = get_all_nodes(exp, t)

# nodes= [
#     node
#     for node in nodes
#     if is_in_ROI_node(node, t) and node.degree(t)==1
# ]
ax = plot_full(exp, t, nodes=nodes,downsizing = 10)


In [ ]:
daad

In [ ]:
fluxes = np.array([edge_flux[edge] for edge in edges if edge.width(t)>2])
radiuses = np.array([edge.width(t)/2 for edge in edges if edge.width(t)>2])
lengths = np.array([edge.length_um(t) for edge in edges if edge.width(t)>2])

speeds = fluxes/(radiuses*2)**2
speeds = speeds[np.where(speeds<50)]
lengths = lengths[np.where(speeds<50)]

fig,ax = plt.subplots()
ax.hist(fluxes*3e-9,30,weights = lengths)

In [ ]:
path[0].begin.neighbours(t)

In [ ]:
edge_flux[Edge(path[0].begin,path[0].end,exp)]

In [ ]:
edge_flux(Edge

In [ ]:
zbzdd

In [ ]:

for edge in vid_obj.edge_objs:
    if "network_begin" in edge.mean_data.keys():
        edge_begin = edge.mean_data["network_begin"]
        edge_end = edge.mean_data["network_end"]    # break
        network_edge = Edge(Node(edge_begin,exp),Node(edge_end,exp),exp) 

In [ ]:
# def get_derivative(edge,t,fun):
# edge = network_edge
fun = lambda edge: edge.betweeness(t)
def get_derivative(edge_d,t,fun):
    edges_begin = edge_d.begin.edges(t)
    edges_begin.remove(edge_d)
    edges_end = edge_d.end.edges(t)
    edges_end.remove(edge_d)
    weight_begin = np.sum([fun(edge) for edge in edges_begin])
    weight_end = np.sum([fun(edge) for edge in edges_end])
    return(weight_end-weight_begin)

In [ ]:
edges_network = []
derivatives = []

coverage_tot = []
for index,vid_obj in enumerate(data_obj.video_objs):
    if vid_obj.dataset['mode']=="BF" and check_hasedges(vid_obj):
        for edge in vid_obj.edge_objs:
            if "network_begin" in edge.mean_data.keys():
                edge_begin = edge.mean_data["network_begin"]
                edge_end = edge.mean_data["network_end"]    # break
                network_edge = Edge(Node(edge_begin,exp),Node(edge_end,exp),exp) 
                if network_edge.is_in(t): 
                    edges_network.append(network_edge)
                    derivatives.append(get_derivative(network_edge,t,fun))

In [ ]:
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plotting import *
downsizing = 5
ax,f = plot_full_video( 
    exp,
    t,
    downsizing=5,
    # points=positions_list,
    video_num=[0],
    edges=edges_network,
    dilation=downsizing,
    # region = region,
    figsize = (18,12),
)
for derivative,edge in zip(derivatives,edges_network):
    pixels = edge.pixel_list(t)
    center_arrow = f(pixels[len(pixels)//2])

    begin_arrow = f(pixels[10])
    end_arrow = f(pixels[-10])
    # begin_arrow = edges_network[0].begin.pos(t)
    # end_arrow = edges_network[0].pixel_list(t)[10]
    vector = (end_arrow-begin_arrow)
    vector = vector/np.linalg.norm(vector)*100/downsizing*(1-2*(derivative>0))
    ax.arrow(center_arrow[1],center_arrow[0],vector[1],vector[0],width = 10)
plt.savefig(f"test_{plate_id_video}_derivative.png", dpi=600)
